In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from plotnine import *

In [ ]:
results_annotated = pd.read_json("../data/results_annotated.jsonl", lines=True)

In [ ]:
results_annotated.head(3).T

In [ ]:
df_pairs = results_annotated.query("label == 'Correct'").pivot(
    index=["model","topic"],
    columns="format",
    values="poem"
).reset_index().dropna(axis=0)

In [ ]:
import pandas as pd
import numpy as np

def shuffle_columns(df, col1, col2):
    # Create a copy of the DataFrame
    df_new = df.copy()

    # Rename the columns
    df_new[col1] = df_new.rename(columns={col1: 'Column1', col2: 'Column2'}, inplace=True)

    # Generate a random shuffling pattern
    choices = np.random.choice([0, 1], size=(len(df_new),))

    # Apply the shuffling pattern
    df_new.loc[choices == 0, 'Column1'] = df[col1]
    df_new.loc[choices == 1, 'Column1'] = df[col2]
    df_new.loc[choices == 0, 'Column2'] = df[col2]
    df_new.loc[choices == 1, 'Column2'] = df[col1]

    return df_new

In [ ]:
df_pairs

In [ ]:
shuffle_columns(df_pairs, "direct_ask", "with_instructor")

In [ ]:
row = df_pairs.sample(1).iloc[0]

row[['direct_ask','with_instructor']].to_list() if random.choice([True, False]) else row[['with_instructor','direct_ask']].values

In [ ]:
choices = np.random.choice([0, 1], size=(len(df_pairs),))

df_pairs.loc[choices == 0, 'poem1'] = df_pairs['direct_ask']
df_pairs.loc[choices == 1, 'poem1'] = df_pairs['with_instructor']
df_pairs.loc[choices == 0, 'poem2'] = df_pairs['with_instructor']
df_pairs.loc[choices == 1, 'poem2'] = df_pairs['direct_ask']

df_pairs['combined_poem'] = df_pairs.apply(lambda row: f"{row['poem1']}'\n---\n'{row['poem2']}", axis=1)


In [ ]:
from abba_eval import pigeonxt
from IPython.display import display
from ipywidgets import HTML

annotations = pigeonxt.annotate(
    df_pairs,
    example_column="combined_poem",
    options=['Poem1', 'Poem2', 'No Preference'],
    display_fn=lambda html: display(HTML(html.value.replace("\n", "<br>")))
)

In [ ]:
(
    ((annotations['label'] == 'Poem1') & (annotations['direct_ask'] == annotations['poem1']))
|
    ((annotations['label'] == 'Poem2') & (annotations['direct_ask'] == annotations['poem2']))
).sum()

In [ ]:
(
    ((annotations['label'] == 'Poem1') & (annotations['with_instructor'] == annotations['poem1']))
|
    ((annotations['label'] == 'Poem2') & (annotations['with_instructor'] == annotations['poem2']))
).sum()

In [ ]:
o = annotations.assign(direct_preferred = (
    ((annotations['label'] == 'Poem1') & (annotations['direct_ask'] == annotations['poem1']))
|
    ((annotations['label'] == 'Poem2') & (annotations['direct_ask'] == annotations['poem2']))
)).assign(
    with_instructor_preferred = (
    ((annotations['label'] == 'Poem1') & (annotations['with_instructor'] == annotations['poem1']))
|
    ((annotations['label'] == 'Poem2') & (annotations['with_instructor'] == annotations['poem2']))
)).assign(no_preference = lambda d: ~(d['direct_preferred'] | d['with_instructor_preferred']))

In [ ]:
o.groupby('model')[['direct_preferred','with_instructor_preferred','no_preference']].sum()


In [ ]:
results_annotated = (
    results
    .merge(annotations[['poem','label']], how="left", on="poem", suffixes=('_x', '_y'))
    .assign(label=lambda x: x['label_y'].combine_first(x['label_x']))
    .drop(columns=['label_x', 'label_y'])
)

results_annotated.to_json("../data/results_annotated.jsonl", lines=True, orient='records')